# [Lab0] 환경 설정 및 초기화

이 노트북에서는 SageMaker 엔드투엔드 머신러닝 워크플로우를 위한 환경을 설정합니다.

## 주요 설정 내용
- AWS 환경 및 권한 설정
- MLflow 추적 서버 연결
- 데이터 준비 및 S3 업로드
- 실험 환경 초기화

## 1. 필수 라이브러리 및 AWS 환경 설정

In [1]:
# 필요한 패키지 설치 (주석 해제하여 실행)
%pip install -q mlflow==2.13.2 sagemaker-mlflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
sagemaker-studio 1.1.4 requires pydynamodb>=0.7.4, which is not installed.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.57.3 which is incompatible.
autogluon-timeseries 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.57.3 which is incompatible.
grpcio-status 1.67.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
sagemaker-studio 1.1.4 requires pyarrow>=19.0.0, but you have pyarrow 15.0.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 패키지 설치 후 커널 재시작 (필요시 주석 해제)
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
# 필수 라이브러리 임포트
import json
import os
import logging
from time import gmtime, strftime

import boto3
import pandas as pd
import numpy as np
import sagemaker
import mlflow

from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker_studio import Project

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


In [3]:
# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 로깅 설정 완료")

✅ 로깅 설정 완료


In [4]:
# AWS 세션 및 SageMaker 설정
boto_session = boto3.Session()
sm_client = boto_session.client("sagemaker")
region = boto_session.region_name
sess = sagemaker.Session()

print(f"✅ AWS 리전: {region}")
print(f"✅ SageMaker 세션 초기화 완료")

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
✅ AWS 리전: us-west-2
✅ SageMaker 세션 초기화 완료


In [6]:


# 현재 세션의 region 사용
session = sagemaker.Session()
region = session.boto_region_name

client = boto3.client('sagemaker', region_name=region)
response = client.list_mlflow_tracking_servers()

mlflow_arn = response['TrackingServerSummaries'][0]['TrackingServerArn']

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


## 2. SageMaker AI 프로젝트 설정

SageMaker AI 프로젝트를 통해 통합된 리소스 관리를 수행합니다.

In [9]:
from sagemaker_studio import Project
import mlflow

# Initialize project (it will auto-detect MLflow if configured)
project = Project()

# Get MLflow ARN from project settings if available
try:
    print(f"   - MLflow 서버: {mlflow_arn}")
    mlflow.set_tracking_uri(mlflow_arn)
except AttributeError:
    # If not configured in project, set it manually
    mlflow_arn = "arn:aws:sagemaker:us-west-2:891377361512:mlflow-tracking-server/tracking-s..."
    mlflow.set_tracking_uri(mlflow_arn)



   - MLflow 서버: arn:aws:sagemaker:us-west-2:975050092050:mlflow-tracking-server/tracking-server-52d9q7u6v2xq8g-as0bts6qd99zww-dev


In [10]:
# S3 버킷 및 IAM 역할 설정
bucket_root = project.s3.root
role = project.iam_role

# S3 URI 파싱
s3_parts = bucket_root.replace("s3://", "").split("/")
bucket = s3_parts[0]
prefix = "/".join(s3_parts[1:] + ["sagemaker/DEMO-xgboost-dm"])

print(f"✅ S3 버킷: {bucket}")
print(f"✅ S3 프리픽스: {prefix}")
print(f"✅ IAM 역할: {role}")

✅ S3 버킷: csv-file-store-b09420d0
✅ S3 프리픽스: dzd-c7j7i2pce12pg0/52d9q7u6v2xq8g/dev/sagemaker/DEMO-xgboost-dm
✅ IAM 역할: arn:aws:iam::975050092050:role/datazone_usr_role_52d9q7u6v2xq8g_6nyh62wzegjdsg


In [11]:
# 프로젝트 데이터베이스 정보 가져오기
catalog = project.connection().catalog()
project_database = catalog.databases[0].name

print(f"✅ 프로젝트 데이터베이스: {project_database}")

✅ 프로젝트 데이터베이스: glue_db_dbjy3mzortm1lc


## 3. 실험 설정

MLflow를 사용한 실험 추적을 위한 설정을 수행합니다.

In [12]:
# 실험 이름 생성
experiment_suffix = strftime('%d-%H-%M-%S', gmtime())
experiment_name = f"end-to-end-experiment-{experiment_suffix}"

print(f"✅ 실험 이름: {experiment_name}")

✅ 실험 이름: end-to-end-experiment-27-02-04-35


## 4. 데이터 준비

은행 마케팅 데이터셋을 준비하고 기본적인 탐색을 수행합니다.

In [13]:
import zipfile

# 데이터 압축 해제
with zipfile.ZipFile('bank-additional.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
    
print("✅ 데이터 압축 해제 완료")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 import zipfile                                                                               │
│   2                                                                                              │
│   3 # 데이터 압축 해제                                                                           │
│ ❱ 4 with zipfile.ZipFile('bank-additional.zip', 'r') as zip_ref:                                 │
│   5 │   zip_ref.extractall('.')                                                                  │
│   6                                                                                              │
│   7 print("✅ 데이터 압축 해제 완료")                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/zipfile.py:1295 in __init__                                            │
│                                                                                                  │
│   1292 │   │   │   filemode = modeDict[mode]                                                     │
│   1293 │   │   │   while True:                                                                   │
│   1294 │   │   │   │   try:                                                                      │
│ ❱ 1295 │   │   │   │   │   self.fp = io.open(file, filemode)                                     │
│   1296 │   │   │   │   except OSError:                                                           │
│   1297 │   │   │   │   │   if filemode in modeDict:                                              │
│   1298 │   │   │   │   │   │   filemode = modeDict[filemode]                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: 'bank-additional.zip'

In [41]:
# 데이터 로드 및 기본 정보 확인
data = pd.read_csv('./bank-additional/bank-additional-full.csv')

print(f"✅ 데이터 로드 완료")
print(f"   - 데이터 크기: {data.shape}")
print(f"   - 컬럼 수: {len(data.columns)}")
print(f"   - 타겟 변수 분포:")
print(data['y'].value_counts())

# 처음 5행 표시
print("\n📊 데이터 미리보기:")
display(data.head())

✅ 데이터 로드 완료
   - 데이터 크기: (41188, 21)
   - 컬럼 수: 21
   - 타겟 변수 분포:
y
no     36548
yes     4640
Name: count, dtype: int64

📊 데이터 미리보기:


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
user_profile_name = os.getenv('USER', 'sagemaker-user')

## 5. 변수 저장

다음 노트북에서 사용할 중요한 변수들을 저장합니다.

In [42]:
# 중요한 변수들을 다음 노트북에서 사용하기 위해 저장
%store bucket
%store prefix
%store role
%store region
%store mlflow_arn
%store mlflow_name
%store experiment_name

print("✅ 모든 변수가 저장되었습니다.")
print("\n📋 저장된 변수 요약:")
print(f"   - S3 버킷: {bucket}")
print(f"   - S3 프리픽스: {prefix}")
print(f"   - 리전: {region}")
print(f"   - 실험 이름: {experiment_name}")
print(f"   - MLflow 서버: {mlflow_name}")

Stored 'bucket' (str)
Stored 'prefix' (str)
Stored 'role' (str)
Stored 'region' (str)
Stored 'mlflow_arn' (str)
Stored 'mlflow_name' (str)
Stored 'experiment_name' (str)
✅ 모든 변수가 저장되었습니다.

📋 저장된 변수 요약:
   - S3 버킷: csv-file-store-6b806890
   - S3 프리픽스: dzd-b6e9451edya6z4/3jxvlofqlqhfv4/dev/sagemaker/DEMO-xgboost-dm
   - 리전: us-west-2
   - 실험 이름: end-to-end-experiment-24-06-16-24
   - MLflow 서버: tracking-server-3jxvlofqlqhfv4-5n1kucn6kabekw-dev


## ✅ 설정 완료

환경 설정이 완료되었습니다! 이제 다음 노트북 `1-preprocessing.ipynb`로 진행하여 데이터 전처리를 수행할 수 있습니다.

### 다음 단계
1. **데이터 전처리** (`1-preprocessing.ipynb`): SageMaker Processing을 사용한 데이터 전처리
2. **모델 훈련** (`2-training.ipynb`): SageMaker Training Job을 사용한 모델 훈련
3. **모델 배포** (`3-test-and-deploy.ipynb`): 모델 평가 및 배포
4. **파이프라인 구축** (`4-pipelines.ipynb`): 엔드투엔드 ML 파이프라인 구축